In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/gdrive


In [10]:
base_folder = "/content/drive/MyDrive/Colab Notebooks/housing_fall2025"
%cd "{base_folder}"

/content/gdrive/MyDrive/Colab Notebooks/housing_fall2025


In [ ]:
import pandas as pd
import sqlite3
import os

def build_churn_database(db_path="project.db"):
    csv_path = "churn_data.csv"
    if not os.path.exists(csv_path):
        csv_path = "../churn_data.csv"
    
    if not os.path.exists(csv_path):
        print("Error: churn_data.csv not found.")
        return

    df = pd.read_csv(csv_path)

    demog_cols = ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents']
    account_cols = [c for c in df.columns if c not in demog_cols or c == 'customerID']

    if os.path.exists(db_path):
        os.remove(db_path)
    
    conn = sqlite3.connect(db_path)

    df[demog_cols].to_sql('demographics', conn, index=False)
    df[account_cols].to_sql('accounts', conn, index=False)

    query = """
    SELECT d.*, a.* 
    FROM demographics d 
    JOIN accounts a ON d.customerID = a.customerID
    """
    
    df_joined = pd.read_sql(query, conn)
    
    df_joined = df_joined.loc[:, ~df_joined.columns.duplicated()]

    df_joined.to_csv("churn_data.csv", index=False)
    
    if os.path.exists("../docker-compose.yml"):
        df_joined.to_csv("../churn_data.csv", index=False)

    print("Database built.")
    print("SQL Join performed.")
    print("Data saved to churn_data.csv ready for Part 2.")
    print(df_joined.head())

    conn.close()

build_churn_database()

=== BUILDING 3NF SQLITE DATA MODEL ===

[STEP 1] Loading CSV into DataFrame…
[1] Checking for housing.tgz…
[1a] File not found. Creating datasets/ and downloading dataset…
[1b] Download completed.
[2] Extracting housing.tgz…
[3] Loading housing.csv into DataFrame…


/tmp/ipython-input-1553684944.py:20: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  housing_tarball.extractall(path="datasets")


Loaded 20640 rows.

[STEP 2] Creating surrogate key block_id…
block_id added.

[STEP 3] Building ocean_proximity dimension table…
Found 5 unique ocean_proximity values.

[STEP 4] Merging ocean_proximity_id into main DataFrame…

[STEP 5] Creating 3NF DataFrames (ocean, block, stats)…
3NF DataFrames created.

[STEP 6] Creating SQLite database and tables…
Running SQL schema creation script…
Tables created.

[STEP 7] Inserting data into SQLite database…
Inserting ocean_proximity dimension…
Inserting block table…
Inserting block_housing_stats…

=== DONE! SQLite DB created at: housing.db ===

